# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [19]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print(f'Count of applicable establishments: {document_count}')

# Check if there are any documents in the results


Count of applicable establishments: 0


In [20]:
# Convert the result to a Pandas DataFrame
hygiene = establishments.find(query)
hygiene_df = pd.DataFrame(hygiene)

# Display the number of rows in the DataFrame
print(hygiene_df.shape[0])

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [22]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName':{'$regex':'London'},'RatingValue': {'$gte':'4'}}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query)) 

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

0
None


In [23]:
# Convert the result to a Pandas DataFrame
london = (establishments.find(query))
london_df = pd.DataFrame(london)

# Display the number of rows in the DataFrame
pprint(london_df.shape[0])

# Display the first 10 rows of the DataFrame
london_df.head(10) 

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {'geocode.latitude':{'$gte':latitude-degree_search,'$lte':latitude+degree_search},
        'geocode.longitude':{'$gte':longitude-degree_search,'$lte':longitude+degree_search},
        'RatingValue':'5'}
sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))

[]


In [11]:
# Convert result to Pandas DataFrame
establishments_1 = list(establishments.find(query).sort(sort).limit(limit))
df_1 = pd.DataFrame(establishments_1)
print(df_1)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
bad_hygiene = {'$match': {'scores.Hygiene':0}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': {"LocalAuthorityName": "$LocalAuthorityName",
                          'count': { '$sum': 1 }
                         }
              }}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': +1 }}
# Print the number of documents in the result
pipeline = [bad_hygiene, group_query, sort_values]
# Print the first 10 results
results = list(establishments.aggregate(pipeline))
# pprint("Number of documents in result: ", len(results))
pprint(results[0:10])

[]


In [13]:
# Convert the result to a Pandas DataFrame
bad_hygiene_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print("Total rows in bad_hygiene_df is ",len(bad_hygiene_df))
# Display the first 10 rows of the DataFrame
bad_hygiene_df.head(10)

Total rows in bad_hygiene_df is  0


""
